In [15]:
import random

In [16]:
def MCD(a, b):
    if b == 0:
        return a
    return MCD(b, a % b)

In [17]:
def  miller_rabin(n: int , k: int) -> bool:
    # Argumentos:
    #    n: int - n >= 1
    #    k: int - k >= 1
    # Retorna:
    #    int - True si n es un  numero  primo , y False en caso  contrario.
    #           La  probabilidad  de  error en el test  debe  ser  menor o
    #           igual a 2**(-k)
    d = n - 1
    r = 0
    while d % 2 == 0:
        d = d // 2
        r += 1
    for _ in range(k):
        next_k = False
        a = random.randint(2, n - 2)
        x = exp_mod(a,d,n)
        if x == 1 or x == n - 1:
            continue
        for _ in range(r - 1):
            x = exp_mod(x,2,n)
            if x == n - 1:
                next_k = True
                break
        if next_k: 
            continue
        return False
    return True

In [18]:
def  generar_primo(l: int) -> int:
    # Argumentos:
    #    l: int - l >= 1
    # Retorna:
    #    int - numero  primo  con al  menos l digitos. La  probabilidad  de
    #    error en la  generacion  debe  ser  menor o igual a 2**(-100)
    while True:
        num = random.randint(10**(l-1), 10**l - 1)
        if miller_rabin(num, 100):
            return num

In [19]:
def  alg_ext_euclides(a: int , b: int) -> (int , int , int):
    # Argumentos :
    #    a: int
    #    b: int - a >= b >= 0 y a > 0
    # Retorna :
    #    (int , int , int) - maximo  comun  divisor  MCD(a,b) entre a y b,
    #    y numeros  enteros s y t tales  que MCD(a,b) = s*a + t*b
    values = {'s_i-1': 1, 's_i': 0, 's_i+1': 0, 't_i-1': 0, 't_i': 1, 
              't_i+1': 0, 'r_i-1': a, 'r_i': b, 'r_i+1': 0}
    while values['r_i'] != 0:
        values['q_i'] = values['r_i-1'] // values['r_i']
        values['r_i+1'] = values['r_i-1'] % values['r_i']
        values['s_i+1'] = values['s_i-1'] - values['q_i']*values['s_i']
        values['t_i+1'] = values['t_i-1'] - values['q_i']*values['t_i']
        values['r_i-1'] = values['r_i']
        values['s_i-1'] = values['s_i']
        values['t_i-1'] = values['t_i']
        values['r_i'] = values['r_i+1']
        values['s_i'] = values['s_i+1']
        values['t_i'] = values['t_i+1']
    return (values['r_i-1'],values['s_i-1'],values['t_i-1'])

In [20]:
def exp_mod(a: int , b: int , n: int) -> int:
    # Argumentos:
    #    a: int - a >= 0
    #    b: int - b >= 0
    #    n: int - n > 0
    # Retorna:
    #    int - a**b en  modulo n
    if b == 1:
        return a % n
    if b % 2 == 0: 
        val = exp_mod(a, b//2, n)
        return val*val % n
    val = exp_mod(a, (b-1)//2, n)
    return val*val*a % n

In [21]:
def inverso(a: int , n: int) -> int:
    # Argumentos:
    #    a: int - a >= 1
    #    n: int - n >= 2, a y n son  primos  relativos
    # Retorna:
    #    int - inverso  de a en  modulo n
    inverso = alg_ext_euclides(n, a)[2]
    if inverso < 0:
        return inverso + n
    return inverso

In [22]:
def  generar_clave(l: int):
    # Argumentos:
    #    l: int - largo de las  claves a ser  generadas
    # Retorna:
    #    genera  una  clave  privada (d,N) y una  clave  publica (e,N) tales
    #    que d, e y N tienen  al  menos l digitos. La  clave  privada  debe
    #    ser  almacenada  en un  archivo  private_key.txt en el  formato:
    #    d
    #    N
    #    La  clave  publica  debe  ser  almacenada  de la  misma  forma en
    #    en un  archivo  public_key.txt
    P = generar_primo((l // 2) + 1)
    Q = generar_primo((l // 2) + 1)
    N = P*Q
    phi_N = (P-1)*(Q-1)

    d = random.randint(3,phi_N)
    while MCD(d, phi_N) != 1:
        d = random.randint(3,phi_N)
        
    e = inverso(d, phi_N)
    
    with open("private_key.txt", "w") as file:
        file.write(str(d))
        file.write('\n')
        file.write(str(N))
        file.write('\n')
    
    with open("public_key.txt", "w") as file:
        file.write(str(e))
        file.write('\n')
        file.write(str(N))
        file.write('\n')

In [23]:
def  enc(m: int) -> int:
    # Argumentos:
    #    m: int - 0 <= m <= N-1, suponiendo  que la  clave  publica
    #    almacenada  en  public_key.txt es (e,N)
    # Retorna:
    #    int: cifrado  de m de  acuerdo  con la  clave  publica  almacenada
    #    en  public_key.txt
    with open("public_key.txt", "r") as file:
        lines = file.readlines()
    e, N = lines[:2]
    return exp_mod(m,int(e),int(N))

In [24]:
def  dec(c: int) -> int:
    # Argumentos:
    #    m: int - 0 <= m <= N-1, suponiendo  que la  clave  privada
    #    almacenada  en  private_key.txt es (d,N)
    # Retorna:
    #    int: descifrado  de m de  acuerdo  con la  clave  privada
    #    almacenada  en  private_key.txt
    with open("private_key.txt", "r") as file:
        lines = file.readlines()
    d, N = lines[:2]
    return exp_mod(c,int(d),int(N))

In [25]:
generar_clave(300)

In [26]:
c = enc(123)

In [27]:
dec(c)

123